# Lecture 4 and 5

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 4.1 FewShotPromptTemplate

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

# prompt.format(country="Germany")

chain.invoke({"country": "Turkey"})



AI: I know this about Turkey:
- Capital: Ankara
- Language: Turkish
- Food: Kebabs and Baklava
- Currency: Turkish Lira

AIMessageChunk(content='AI: I know this about Turkey:\n- Capital: Ankara\n- Language: Turkish\n- Food: Kebabs and Baklava\n- Currency: Turkish Lira')

## 4.2 FewShotChatMessagePromptTemplate

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography export, you give short answer"),
        example_prompt, 
        ("human","What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Germany"})



        I know this:
        Capital: Berlin
        Language: German
        Food: Sausages and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Sausages and Sauerkraut\n        Currency: Euro\n        ')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, F
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

## 4.3 LengthBasedExampleSelector

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# chain = prompt | chat
# chain.invoke({"country": "Turkey"})

prompt.format(country="Brazil")


'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## 4.4 Serialization and Composition

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

# 롤플레잉 도우미이고 캐릭터를 흉내내는 AI
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

# 캐릭터들이 어떻게 질문하고 답하는지에 대한 형태 
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

# AI도우미가 우리의 텍스트를 완성해주는 start 예제 
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 이 모든것을 하나로 합친 프롬프트 
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' rum! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!

AIMessageChunk(content="Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' rum! Nothin' like a good feast after a long day o' plunderin' the high seas! Arg arg!")

## Caching

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamlitCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

# 모든 response가 메모리에 저장됨 
set_llm_cache(InMemoryCache())

# 무슨일을 하고 있는지 보여줌.
# 추후 체인작업을 수행할때 도움이 됨 
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("how do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: how do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.91s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and rol

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a rolling pin or pasta machine.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for a few minutes until 

In [26]:
chat.predict("how do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamlitCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# 모든 response가 메모리에 저장됨 
set_llm_cache(SQLiteCache("cache.db"))

# 무슨일을 하고 있는지 보여줌.
# 추후 체인작업을 수행할때 도움이 됨 
set_debug(False)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("how do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n10. Serve hot and enjoy your homemade Italian pasta!'

## 4.6 Serialization

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a,b,"\n")
    print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and yeast with water until dissolved.
5. Add the cooked rice to the bowl and mix well.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.
9. Serve the homemade soju chilled and enjoy responsibly. Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil

Instructions:
1. In a small bowl, combine the warm water, sugar, and yeast.

In [37]:
# Serialization (저장)
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    model='gpt-4o-mini', 
    temperature=0.1,
    max_tokens=450
)

chat.save("model.json")

In [38]:
# Serialization (불러오기)
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json")
chat



/Users/hwangms/Documents/workspace/LLM_Study/langchain_study/hwang/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/hwangms/Documents/workspace/LLM_Study/langchain_study/hwang/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4o-mini', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

## 5.0 ConversationBufferMemory

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [3]:
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})


{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [4]:
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

## 5.1 ConversationBufferWindowMemory

In [5]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4   # 대화 내용을 몇개나 저장할지?
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})
    
add_message(1,1)

In [6]:
add_message(2,2)
add_message(3,3)
add_message(4,4)
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [7]:

add_message(5,5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

## 5.2 ConversationSummaryMemory

In [10]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Hwang, I live in South Korea", "Wow that is so cool!")

In [11]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [12]:
get_history()

결과 : 
{'history': 'The human introduces themselves as Hwang from South Korea, \
    and the AI responds positively, expressing excitement about this \
    information. The human comments on the beauty of South Korea, \
    and the AI expresses a wish to visit.'}

{'history': 'The human introduces themselves as Hwang from South Korea, and the AI responds positively, expressing excitement about this information. The human comments on the beauty of South Korea, and the AI expresses a wish to visit.'}

## 5.3 ConversationSummaryBufferMemory

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Hwang, I live in South Korea", "Wow that is so cool!")

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/o200k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

## 5.4 ConversationKGMemory

In [3]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Hwang, I live in South Korea", "Wow that is so cool!")

In [4]:
memory.load_memory_variables({"input":"who is Hwang"})

{'history': [SystemMessage(content='On Hwang: Hwang lives in South Korea.')]}

In [5]:
add_message("Hwang likes kimchi!", "Wow that is so cool!")

In [6]:
memory.load_memory_variables({"input":"what does Hwang like"})

{'history': [SystemMessage(content='On Hwang: Hwang lives in South Korea. Hwang likes kimchi.')]}

## 5.5 Memory on LLMChain

In [29]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

# 오래된 대화내용은 지워지고 최신 대화내용만 저장 
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"   # 이거 추가로 memory.load_memory_variables({}) 할 필요 없음 
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human:{question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, # chain의 프롬프트 로그들을 확인할 수 있음 
)

chain.predict(question="My name is Hwang")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human:My name is Hwang
    You: 


> Finished chain.


"Hello, Hwang! It's nice to meet you. How can I assist you today?"

In [30]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Hwang
AI: Hello, Hwang! It's nice to meet you. How can I assist you today?
    Human:I live in Seoul
    You: 


> Finished chain.


"That's great, Hwang! Seoul is a vibrant city with a rich history and culture. What do you enjoy most about living there?"

In [ ]:
chain.predict(question="What is my name?")

In [ ]:
# 이름을 위에 말했음에도 모른다고 답변을 준 이유는 기존 대화 내역을 말해주지 않았기 때문임
# 하지만, memory를 확인해보면 내용은 저장중임 (memory.load_memory_variables({}))
# 즉, memory에 담긴 내용을 프롬프트에 포함하지 않았기 때문에 이런 결과를 초래한 것
# 그러므로 아래 코드처럼 template을 만들고, memory_key를 추가한 후 chain 내 Prompt에 추가하면 됨 

# 오래된 대화내용은 지워지고 최신 대화내용만 저장 
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"   # 이거 추가로 memory.load_memory_variables({}) 할 필요 없음 
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human:{question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, # chain의 프롬프트 로그들을 확인할 수 있음 
)

chain.predict(question="My name is Hwang")


In [38]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human introduces themselves as Hwang, and the AI responds warmly, expressing pleasure in meeting Hwang and asking how it can assist them today. Hwang mentions that they live in Seoul.
AI: That's great, Hwang! Seoul is a vibrant city with a rich history and culture. What do you enjoy most about living there?
Human: What's your name?
AI: AI: I don't have a personal name like you do, but you can call me Assistant! How can I help you further, Hwang?
Human: What's my name?
AI: Your name is Hwang!
Human: What's my name?
AI: Your name is still Hwang! How can I assist you today?
    Human:I live in Seoul
    You: 


> Finished chain.


"That's wonderful, Hwang! Seoul has so much to offer, from its delicious food to its beautiful landmarks. Is there something specific about Seoul that you'd like to talk about or explore today?"

In [39]:
chain.predict(question="What's my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human introduces themselves as Hwang, and the AI responds warmly, expressing pleasure in meeting Hwang and asking how it can assist them today. Hwang mentions that they live in Seoul. The AI comments on Seoul's vibrancy and asks what Hwang enjoys most about living there. Hwang then inquires about the AI's name, to which the AI explains it doesn't have a personal name but can be called Assistant, and asks how it can help further.
Human: What's my name?
AI: Your name is Hwang!
Human: What's my name?
AI: Your name is still Hwang! How can I assist you today?
Human: I live in Seoul
AI: That's wonderful, Hwang! Seoul has so much to offer, from its delicious food to its beautiful landmarks. Is there something specific about Seoul that you'd like to talk about or explore today?
    Human:What's my name?
    You: 


> Finished chain.


'Your name is Hwang! How can I assist you further today?'

In [37]:
memory.load_memory_variables({})

{'chat_history': "System: The human introduces themselves as Hwang, and the AI responds warmly, expressing pleasure in meeting Hwang and asking how it can assist them today. Hwang mentions that they live in Seoul.\nAI: That's great, Hwang! Seoul is a vibrant city with a rich history and culture. What do you enjoy most about living there?\nHuman: What's your name?\nAI: AI: I don't have a personal name like you do, but you can call me Assistant! How can I help you further, Hwang?\nHuman: What's my name?\nAI: Your name is Hwang!\nHuman: What's my name?\nAI: Your name is still Hwang! How can I assist you today?"}

## 5.6 Chat Based Memory

In [40]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

# 오래된 대화내용은 지워지고 최신 대화내용만 저장 
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",   # 이거 추가로 memory.load_memory_variables({}) 할 필요 없음 
    return_messages=True
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human:{question}
    You: 
"""

# ConversationSummaryBufferMemory가 AI, 사람, 시스템 메시지를 줄거고, 
# 얼마나 많은 내용이 있을지 알 수 없기 때문에 MessagesPlaceholder를 사용함.
# 즉, 메시지가 얼마나 많고, 누구에게로부터 왔는지 모르지만 이 모든것들은 Memory class로 대체할거란 뜻 
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    # prompt=PromptTemplate.from_template(template),
    prompt=prompt, # 위의 PromptTemplate은 더 위의 MessagesPlaceholder로 대체하여 사용 
    verbose=True, # chain의 프롬프트 로그들을 확인할 수 있음 
)

chain.predict(question="My name is Hwang")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Hwang

> Finished chain.


'Nice to meet you, Hwang! How can I assist you today?'

In [41]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Hwang
AI: Nice to meet you, Hwang! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant city with a rich history and a lot to offer. Do you have any favorite places or activities in Seoul?"

In [42]:
chain.predict(question="What's my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Hwang
AI: Nice to meet you, Hwang! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant city with a rich history and a lot to offer. Do you have any favorite places or activities in Seoul?
Human: What's my name?

> Finished chain.


'Your name is Hwang. How can I assist you further?'

## 5.7 LCEL Based Memory

In [49]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1,    
    model_name='gpt-4o-mini', 
)

# 오래된 대화내용은 지워지고 최신 대화내용만 저장 
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history",   # 사실 default가 hitory이기 때문에 이 라인은 불필요함 
    return_messages=True
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human:{question}
    You: 
"""

# ConversationSummaryBufferMemory가 AI, 사람, 시스템 메시지를 줄거고, 
# 얼마나 많은 내용이 있을지 알 수 없기 때문에 MessagesPlaceholder를 사용함.
# 즉, 메시지가 얼마나 많고, 누구에게로부터 왔는지 모르지만 이 모든것들은 Memory class로 대체할거란 뜻 
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":"My name is Hwang"})
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


In [50]:
invoke_chain("My name is Hwang")

content='Nice to meet you, Hwang! How can I assist you today?'


In [51]:
invoke_chain("What is my name?")

content='Got it, Hwang! What would you like to talk about today?'


## 5.8 Recap